In [1]:
# Import 
import os
import numpy as np
import glob
import PIL.Image as Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


torch.cuda.empty_cache()

# Get CUDA
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# Define Hotdog data class
class Hotdog_NotHotdog(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path='/dtu/datasets1/02514/hotdog_nothotdog'):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path +'/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: id for id, c in enumerate(image_classes)}
        self.image_paths = glob.glob(data_path + '/*/*.jpg')
        
    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]
        
        image = Image.open(image_path)
        c = os.path.split(os.path.split(image_path)[0])[1]
        y = self.name_to_label[c]
        X = self.transform(image)
        return X, y



# Load data
size = 128
train_transform = transforms.Compose([transforms.Resize((size, size)), 
                                    transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((size, size)), 
                                    transforms.ToTensor()])

batch_size = 16
trainset = Hotdog_NotHotdog(train=True, transform=train_transform)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=3)
testset = Hotdog_NotHotdog(train=False, transform=test_transform)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=3)

#We define the training as a function so we can easily re-use it.
def train(model, optimizer, num_epochs=10):
    def loss_fun(output, target):
        return F.cross_entropy(output, target)  #torch.log()
    out_dict = {'train_acc': [],
              'test_acc': [],
              'train_loss': [],
              'test_loss': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        #For each epoch
        train_correct = 0
        train_loss = []
        for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            data, target = data.to(device), target.to(device)
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            output = model(data)
            #Compute the loss
            loss = loss_fun(output, target)
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()

            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = output.argmax(1)
            train_correct += (target==predicted).sum().cpu().item()
        #Comput the test accuracy
        test_loss = []
        test_correct = 0
        model.eval()
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
            test_loss.append(loss_fun(output, target).cpu().item())
            predicted = output.argmax(1)
            test_correct += (target==predicted).sum().cpu().item()
        out_dict['train_acc'].append(train_correct/len(trainset))
        out_dict['test_acc'].append(test_correct/len(testset))
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['test_loss'].append(np.mean(test_loss))
        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%")
    return out_dict

class ResNetBlock(nn.Module):
    def __init__(self, n_features):
        super(ResNetBlock, self).__init__()
        self.block = nn.Sequential(      
            nn.Conv2d(in_channels=n_features, out_channels=n_features, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=n_features, out_channels=n_features, kernel_size=3, padding=1, stride=1))
        
    def forward(self, x):
        Fx = self.block(x)
        out = Fx + x 
        out = F.relu(out)
        return out




class ResNet(nn.Module):
    def __init__(self, n_in, n_features, num_res_blocks=3):
        super(ResNet, self).__init__()
        #First conv layers needs to output the desired number of features.
        conv_layers = [nn.Conv2d(n_in, n_features, kernel_size=3, stride=1, padding=1),
                       nn.BatchNorm2d(n_features),  nn.ReLU()]
        for i in range(num_res_blocks):
            conv_layers.append(ResNetBlock(n_features))
            conv_layers.append(nn.BatchNorm2d(n_features))
        self.res_blocks = nn.Sequential(*conv_layers)
        
        self.fc = nn.Sequential(
                                
                                nn.Linear(size*size*n_features, 2048*2),
                                nn.BatchNorm1d(2048*2),
                                nn.ReLU(),
                                nn.Dropout(0.5),

                                nn.Linear(2048*2, 512*2),
                                nn.BatchNorm1d(512*2),
                                nn.ReLU(),
                                nn.Dropout(0.5),

                                nn.Linear(512*2, 512),
                                nn.BatchNorm1d(512),
                                nn.ReLU(),
                                nn.Dropout(0.5),
                                
                                nn.Linear(512,2))

        
    def forward(self, x):
        x = self.res_blocks(x)
        #reshape x so it becomes flat, except for the first dimension (which is the minibatch)
        x = x.view(x.size(0), -1)
        out = self.fc(x)
        return out #F.log_softmax(out, dim=1)


model = ResNet(n_in=3, n_features=16, num_res_blocks=4)
model.to(device)
#Initialize the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
out_dict = train(model, optimizer)

The code will run on GPU.


 10%|████████                                                                         | 1/10 [00:11<01:46, 11.78s/epoch]

Loss train: 0.637	 test: 0.574	 Accuracy train: 63.8%	 test: 73.0%



 20%|████████████████▏                                                                | 2/10 [00:23<01:33, 11.74s/epoch]

Loss train: 0.536	 test: 0.552	 Accuracy train: 73.3%	 test: 72.9%


...

Loss train: 0.268	 test: 0.525	 Accuracy train: 90.1%	 test: 76.0%



 90%|████████████████████████████████████████████████████████████████████████▉        | 9/10 [01:44<00:11, 11.56s/epoch]

Loss train: 0.238	 test: 0.559	 Accuracy train: 91.7%	 test: 75.2%



100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [01:55<00:00, 11.60s/epoch]

Loss train: 0.212	 test: 0.542	 Accuracy train: 92.2%	 test: 75.3%


In [2]:
out_dict

{'train_acc': [0.6375183194919394,
  0.7332681973619931,
  0.7860283341475329,
  0.8212017586712261,
  0.8334147532975086,
  0.857352222765022,
  0.8827552515876893,
  0.9013190034196384,
  0.9169516365412799,
  0.9223253541768441],
 'test_acc': [0.7303974221267454,
  0.7287862513426423,
  0.7438238453276047,
  0.7561761546723953,
  0.7588614393125671,
  0.7491944146079484,
  0.7545649838882922,
  0.7604726100966702,
  0.7524167561761547,
  0.7529538131041891],
 'train_loss': [0.637345562223345,
  0.5363712611142546,
  0.4759537624195218,
  0.4200290624285117,
  0.3882723735878244,
  0.35460593982134014,
  0.2988264929736033,
  0.2679946622811258,
  0.23836680653039366,
  0.21186114998999983],
 'test_loss': [0.5742789496723403,
  0.5515202437965279,
  0.5394894282023112,
  0.5255774558227286,
  0.5244137232120221,
  0.5290570836036633,
  0.5379564449445814,
  0.5252362157608199,
  0.5588298009183162,
  0.542495295819309]}